In [149]:
import os
import pandas as pd
from kgtk.configure_kgtk_notebooks import ConfigureKGTK

In [150]:
input_path = "/Volumes/saggu-ssd/wikidata-dwd-v2"
output_path = "/Volumes/saggu-ssd/wikidata-dwd-v2"

project_name = "kgtk-search"

files = 'label,pagerank_undirected,alias'

In [151]:
files = files.split(',')

In [152]:
ck = ConfigureKGTK(files)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name)

User home: /Users/amandeep
Current dir: /Users/amandeep/Github/kgtk-browser
KGTK dir: /Users/amandeep/Github
Use-cases dir: /Users/amandeep/Github/use-cases


In [153]:
ck.print_env_variables()

KGTK_OPTION_DEBUG: false
EXAMPLES_DIR: /Users/amandeep/Github/examples
KGTK_GRAPH_CACHE: /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search/temp.kgtk-search/wikidata.sqlite3.db
TEMP: /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search/temp.kgtk-search
KGTK_LABEL_FILE: /Volumes/saggu-ssd/wikidata-dwd-v2/labels.en.tsv.gz
kgtk: kgtk
OUT: /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search
GRAPH: /Volumes/saggu-ssd/wikidata-dwd-v2
USE_CASES_DIR: /Users/amandeep/Github/use-cases
kypher: kgtk query --graph-cache /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search/temp.kgtk-search/wikidata.sqlite3.db
STORE: /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search/temp.kgtk-search/wikidata.sqlite3.db
label: /Volumes/saggu-ssd/wikidata-dwd-v2/labels.en.tsv.gz
pagerank_undirected: /Volumes/saggu-ssd/wikidata-dwd-v2/metadata.pagerank.undirected.tsv.gz
alias: /Volumes/saggu-ssd/wikidata-dwd-v2/aliases.en.tsv.gz


In [154]:
ck.load_files_into_cache()

kgtk query --graph-cache /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search/temp.kgtk-search/wikidata.sqlite3.db -i "/Volumes/saggu-ssd/wikidata-dwd-v2/labels.en.tsv.gz" --as label  -i "/Volumes/saggu-ssd/wikidata-dwd-v2/metadata.pagerank.undirected.tsv.gz" --as pagerank_undirected  -i "/Volumes/saggu-ssd/wikidata-dwd-v2/aliases.en.tsv.gz" --as alias  --limit 3
id	node1	label	node2
P10-label-en	P10	label	'video'@en
P1000-label-en	P1000	label	'record held'@en
P1001-label-en	P1001	label	'applies to jurisdiction'@en


## Create a text search index on labels file

In [168]:
%%time
!kgtk --debug query -i label --idx text:node2//name=labidx --gc $STORE --limit 5

[2021-11-18 11:20:25 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2021-11-18 11:20:25 sqlstore]: CREATE VIRTUAL TABLE "graph_1_txtidx_labidx" USING FTS5 ("node2", tokenize="trigram", content="graph_1")
[2021-11-18 11:20:25 sqlstore]: INSERT INTO "graph_1_txtidx_labidx" ("node2") SELECT "node2" FROM graph_1
id	node1	label	node2
P10-label-en	P10	label	'video'@en
P1000-label-en	P1000	label	'record held'@en
P1001-label-en	P1001	label	'applies to jurisdiction'@en
P1002-label-en	P1002	label	'engine configuration'@en
P1003-label-en	P1003	label	'National Library of Romania ID'@en
CPU times: user 21.3 s, sys: 7.35 s, total: 28.7 s
Wall time: 17min 22s


## Create a text search index on aliases file

In [169]:
%%time
!kgtk --debug query -i alias --idx text:node2//name=aliasidx --gc $STORE --limit 5

[2021-11-18 11:37:48 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_3 AS graph_3_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2021-11-18 11:37:48 sqlstore]: CREATE VIRTUAL TABLE "graph_3_txtidx_aliasidx" USING FTS5 ("node2", tokenize="trigram", content="graph_3")
[2021-11-18 11:37:48 sqlstore]: INSERT INTO "graph_3_txtidx_aliasidx" ("node2") SELECT "node2" FROM graph_3
id	node1	label	node2
P10-alias-en-282226-0	P10	alias	'gif'@en
P10-alias-en-2f86d8-0	P10	alias	'animation'@en
P10-alias-en-c1427e-0	P10	alias	'media'@en
P10-alias-en-c61ab1-0	P10	alias	'trailer (Commons)'@en
P1001-alias-en-0dd7ce-0	P1001	alias	'belongs to jurisdiction'@en
CPU times: user 3.15 s, sys: 1.09 s, total: 4.24 s
Wall time: 2min 33s


## Create a value index on pagerank file

In [170]:
%%time
!kgtk --debug query -i pagerank_undirected --comment 'undirected pagerank' --idx mode:valuegraph --gc $STORE --limit 5

[2021-11-18 11:40:22 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_2 AS graph_2_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2021-11-18 11:40:22 sqlstore]: CREATE INDEX "graph_2_node1_idx" ON "graph_2" ("node1")
[2021-11-18 11:44:39 sqlstore]: ANALYZE "graph_2_node1_idx"
node1	label	node2	id
P10	vertex_in_degree	0	P10-vertex_in_degree-0
P10	vertex_out_degree	11	P10-vertex_out_degree-1
P10	undirected_pagerank	1.7127847966708486e-08	P10-undirected_pagerank-2
Q34508	vertex_in_degree	0	Q34508-vertex_in_degree-3
Q34508	vertex_out_degree	449	Q34508-vertex_out_degree-4
CPU times: user 6.07 s, sys: 2.13 s, total: 8.2 s
Wall time: 4min 42s


In [171]:
!kgtk query --gc $STORE --show-cache 

Graph Cache:
DB file: /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search/temp.kgtk-search/wikidata.sqlite3.db
  size:  23.00 GB   	free:  0 Bytes   	modified:  2021-11-18 11:45:00

KGTK File Information:
alias:
  size:  130.57 MB   	modified:  2021-11-17 14:57:10   	graph:  graph_3
label:
  size:  590.02 MB   	modified:  2021-11-17 14:57:46   	graph:  graph_1
pagerank_undirected:
  size:  1.53 GB   	modified:  2021-11-17 14:58:47   	graph:  graph_2
  comment:  undirected pagerank

Graph Table Information:
graph_1:
  size:  6.56 GB   	created:  2021-11-18 10:52:24
  header:  ['id', 'node1', 'label', 'node2']
graph_2:
  size:  15.24 GB   	created:  2021-11-18 11:18:35
  header:  ['node1', 'label', 'node2', 'id']
graph_3:
  size:  1.20 GB   	created:  2021-11-18 11:19:50
  header:  ['id', 'node1', 'label', 'node2']


In [172]:
%%time
!kgtk query --gc $STORE \
       -i label pagerank_undirected \
       --match 'label: (qnode)-[]->(y), pagerank: (qnode)-[:undirected_pagerank]->(pr)' \
       --where 'textmatch(y, "sch arn")' \
       --return 'qnode as node1, y as label, matchscore(y) as score, cast(pr, float) as prank' \
       --order 'score * prank' \
       --limit 10

node1	label	score	prank
Q154770	'Arnold Schoenberg'@en	-10.783724826293188	3.0558750084414733e-07
Q2685	'Arnold Schwarzenegger'@en	-10.080805077230513	2.918561446324341e-07
Q506494	'Schäftlarn'@en	-12.2824955145365	1.9308762740143706e-07
Q668095	'Scharnstein'@en	-12.043375147640235	1.9003687268452937e-07
Q664826	'Scharnitz'@en	-12.53130366790072	1.3360516624002156e-07
Q522320	'Arnschwang'@en	-12.2824955145365	1.2374568456115837e-07
Q5659790	'Harnischia'@en	-12.2824955145365	1.0656777114445524e-07
Q2667816	'Scharnegoutum'@en	-11.592019328049465	1.118220599905683e-07
Q77325	'Arno Schmidt'@en	-11.813387555683887	9.8642136558971e-08
Q509737	'Scharnhorst Order'@en	-10.783724826293188	9.370581553960886e-08
CPU times: user 7.93 s, sys: 2.79 s, total: 10.7 s
Wall time: 5min 57s


In [173]:
%%time
!kgtk query --gc $STORE \
       -i label pagerank_undirected \
       --match 'label: (qnode)-[]->(y), pagerank: (qnode)-[:undirected_pagerank]->(pr)' \
       --where 'textmatch(y, "arn sch")' \
       --return 'qnode as node1, y as label, matchscore(y) as score, cast(pr, float) as prank' \
       --order 'score * prank' \
       --limit 10

node1	label	score	prank
Q154770	'Arnold Schoenberg'@en	-10.783724826293188	3.0558750084414733e-07
Q2685	'Arnold Schwarzenegger'@en	-10.080805077230513	2.918561446324341e-07
Q506494	'Schäftlarn'@en	-12.2824955145365	1.9308762740143706e-07
Q668095	'Scharnstein'@en	-12.043375147640235	1.9003687268452937e-07
Q664826	'Scharnitz'@en	-12.53130366790072	1.3360516624002156e-07
Q522320	'Arnschwang'@en	-12.2824955145365	1.2374568456115837e-07
Q5659790	'Harnischia'@en	-12.2824955145365	1.0656777114445524e-07
Q2667816	'Scharnegoutum'@en	-11.592019328049465	1.118220599905683e-07
Q77325	'Arno Schmidt'@en	-11.813387555683887	9.8642136558971e-08
Q509737	'Scharnhorst Order'@en	-10.783724826293188	9.370581553960886e-08
CPU times: user 53.3 ms, sys: 32.2 ms, total: 85.5 ms
Wall time: 2.74 s


In [174]:
%%time
!kgtk query --gc $STORE \
       -i alias pagerank_undirected \
       --match 'alias: (qnode)-[]->(y), pagerank: (qnode)-[:undirected_pagerank]->(pr)' \
       --where 'textmatch(y, "sch arn")' \
       --return 'distinct qnode as node1, y as label, matchscore(y) as score, cast(pr, float) as prank' \
       --order 'score * prank' \
       --limit 10

node1	label	score	prank
Q154770	'Arnold Schonberg'@en	-11.359223668032428	3.0558750084414733e-07
Q154770	'Arnold Schönberg'@en	-11.359223668032428	3.0558750084414733e-07
Q660390	'Karnische Alpen'@en	-11.595664671317598	2.6557908090413596e-07
Q154770	'Arnold Franz Walter Schönberg'@en	-8.97908360989274	3.0558750084414733e-07
Q2685	'Arnold Alois Schwarzenegger'@en	-9.278174398231696	2.918561446324341e-07
Q154770	'Arnold Franz Walter Schoenberg'@en	-8.8366546209866	3.0558750084414733e-07
Q98690549	'Deutschsprachige Literatur von Migrantinnen aus Österreich-Ungarn 1867–1918'@en	-5.156164512030479	2.846958102002629e-07
Q522803	'Günther Edward Arnold Schneider'@en	-8.698673582277555	1.4083329835960302e-07
Q77325	'Arno Otto Schmidt'@en	-11.132232267306927	9.8642136558971e-08
Q7895890	'Nebraska State Normal School at Kearney'@en	-7.732724857432492	1.1446407340709346e-07
CPU times: user 201 ms, sys: 78.1 ms, total: 279 ms
Wall time: 11.2 s


In [175]:
def search_labels(search_text) -> pd.DataFrame:
    command = f"""kgtk query --gc {os.environ['STORE']} \
       -i label pagerank_undirected \
       --match 'label: (qnode)-[]->(y), pagerank: (qnode)-[:undirected_pagerank]->(pr)' \
       --where 'textmatch(y, "{search_text}")' \
       --return 'distinct qnode as node1, y as label, 10*matchscore(y) as score, cast(pr, float) as prank' \
       --order-by 'score*prank' \
       --limit 10"""
    results = !$command
    return pd.DataFrame([x.split('\t') for x in results[1:]], columns=results[0].split('\t'))

In [176]:
search_labels('arn schw')

,node1,label,score,prank
0,Q2685,'Arnold Schwarzenegger'@en,-132.7711962060187,2.918561446324341e-07
1,Q522320,'Arnschwang'@en,-161.7689866401114,1.2374568456115837e-07
2,Q57492,'Friedrich Karl Arnold Schwassmann'@en,-111.05447457472671,6.236491910269596e-08
3,Q692078,'Arndt Schwering-Sohnrey'@en,-128.5805384653808,4.9753478733880267e-08
4,Q10777734,'Schweinerden/Swinjarnja'@en,-128.5805384653808,2.7834832014702904e-08
5,Q90957,"'Günther XLI, Count of Schwarzburg-Arnstadt'@en",-98.91963186856057,3.230711790580684e-08
6,Q125639,'Arnold Gerschwiler'@en,-139.5956863326536,2.2378022526459618e-08
7,Q55078379,'Arnold Schwartz'@en,-147.1597538081265,2.0088428460721015e-08
8,Q696162,'Arnold Schwellensattl'@en,-132.7711962060187,2.1994551706639098e-08
9,Q75457075,'Isobel Joyce Schwartz Barnes'@en,-119.17660911611748,2.1596893815515037e-08


In [179]:
def search_aliases(search_text) -> pd.DataFrame:
    command = f"""kgtk query --gc {os.environ['STORE']} \
       -i alias pagerank_undirected \
       --match 'alias: (qnode)-[]->(y), pagerank: (qnode)-[:undirected_pagerank]->(pr)' \
       --where 'textmatch(y, "{search_text}")' \
       --return 'distinct qnode as node1, y as label, matchscore(y) as score, cast(pr, float) as prank' \
       --order-by 'score*prank' \
       --limit 10"""
    results = !$command
    return pd.DataFrame([x.split('\t') for x in results[1:]], columns=results[0].split('\t'))

In [180]:
search_aliases('arn schw')

,node1,label,score,prank
0,Q2685,'Arnold Alois Schwarzenegger'@en,-11.654930057328915,2.918561446324341e-07
1,Q57492,'Arnold Schwassmann'@en,-13.709968734786349,6.236491910269596e-08
2,Q901541,'Patrick Arnold Shriver Schwarzenegger'@en,-9.990951002543062,5.935643959273848e-08
3,Q59152486,'Ursula Tarnowski Schwerin'@en,-12.056529723428401,3.355065040678388e-08
4,Q4710073,'Albert Earnest Schwab'@en,-12.948903195253443,2.5035151616998635e-08
5,Q14094465,'Arnošt Schwarzenberg'@en,-13.193026232585098,2.1732118570625047e-08
6,Q94627760,'Carlos Arnaldo Schwantes'@en,-12.267890089359287,2.2408265274195712e-08
7,Q14094465,'Arnošt ze Schwarzenbergu'@en,-12.267890089359287,2.1732118570625047e-08
8,Q55078379,'Arnold Bernhard Schwartz'@en,-12.267890089359287,2.0088428460721015e-08
9,Q22102137,'Arnold Martin Schwartzman'@en,-12.056529723428401,1.8039107275887433e-08


In [181]:
def search(search_text) -> pd.DataFrame:
    df_labels = search_labels(search_text)
    df_aliases = search_aliases(search_text)
    df = pd.concat([df_aliases, df_labels])
    df['computed_score'] = df.apply(lambda x: float(x.score) * float(x.prank), axis=1)
    return df.sort_values(by=['computed_score'], ascending=True).drop_duplicates(subset=['node1'])
    

In [182]:
search('arn schw')

,node1,label,score,prank,computed_score
0,Q2685,'Arnold Schwarzenegger'@en,-132.7711962060187,2.918561446324341e-07,-3.875009e-05
1,Q522320,'Arnschwang'@en,-161.7689866401114,1.2374568456115837e-07,-2.001821e-05
2,Q57492,'Friedrich Karl Arnold Schwassmann'@en,-111.05447457472671,6.236491910269596e-08,-6.925903e-06
3,Q692078,'Arndt Schwering-Sohnrey'@en,-128.5805384653808,4.9753478733880267e-08,-6.397329e-06
4,Q10777734,'Schweinerden/Swinjarnja'@en,-128.5805384653808,2.7834832014702904e-08,-3.579018e-06
5,Q90957,"'Günther XLI, Count of Schwarzburg-Arnstadt'@en",-98.91963186856057,3.230711790580684e-08,-3.195808e-06
6,Q125639,'Arnold Gerschwiler'@en,-139.5956863326536,2.2378022526459618e-08,-3.123875e-06
7,Q55078379,'Arnold Schwartz'@en,-147.1597538081265,2.0088428460721015e-08,-2.956208e-06
8,Q696162,'Arnold Schwellensattl'@en,-132.7711962060187,2.1994551706639098e-08,-2.920243e-06
9,Q75457075,'Isobel Joyce Schwartz Barnes'@en,-119.17660911611748,2.1596893815515037e-08,-2.573845e-06


In [183]:
search('bara oba')

,node1,label,score,prank,computed_score
0,Q59661289,'Barack Obama Presidential Library'@en,-113.07325551305979,8.33424595681149e-06,-0.000942
1,Q76,'Barack Obama'@en,-158.41887948122047,2.063783996387323e-06,-0.000327
2,Q502486,'Mobara'@en,-178.91939929029448,2.199364095266067e-07,-0.000039
3,Q498027,'Nasushiobara'@en,-158.41887948122047,1.9515463994239617e-07,-0.000031
4,Q14507940,'Asobara'@en,-175.1419651262718,1.253228585856236e-07,-0.000022
5,Q15715887,'Mobarakabad Rural District'@en,-124.99985695546866,9.523858886592924e-08,-0.000012
6,Q81440,'Mobarakeh'@en,-168.0462167308297,6.779914618952456e-08,-0.000011
7,Q1205581,'Kitashiobara'@en,-158.41887948122047,6.784457350233262e-08,-0.000011
8,Q5677233,'Mobarakeh Rural District'@en,-128.88393136038536,7.562824093184876e-08,-0.000010
9,Q81443,'Mobarakeh County'@en,-147.1765862690839,6.4115717704689e-08,-0.000009
